### 读入tools 生成的possible matchings

In [458]:
# matchings_path = "employee.json"
# answers_path = "employee_ans.json"
matchings_path = "./dataset/author.json"
answers_path = "./dataset/author_ans.json"
# matchings_path = "purchase.json"
# answers_path = "purchase_ans.json"
dataset_name = "author"

In [459]:
from fact import FactSet
from llm_api import gpt_check
import numpy as np
import json

with open(matchings_path, "r") as f:
    content = json.load(f)

### 初始化 chatgpt 的 tokenzier，用于估计correspondence 的cost func

In [460]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

### process 统计，factset 需要的数据

In [461]:
def process(content):
    import numpy as np
    c_set = content["correspondence_set"]
    # chatgpt 的tokens 的num list
    len_list = [len(encoding.encode(i[0][1]+i[1][1])) for i in c_set]
    matchings = content["matchings"]
    Views = []
    for match in matchings:
        view = []
        for c in c_set:
            if c in match:
                view.append(1)
            else:
                view.append(0)
        Views.append(view)
    return np.array(Views), np.array(content["prob_all"]), np.array(len_list), max(len_list), sum(len_list), c_set

### 处理后生成 facts 

In [462]:
facts, prob, len_list, max_cost, sum_c, c_set = process(content)
print("max_cost: ", max_cost, f"note that budget of each round must > {3*max_cost} and < {sum_c}:")
ex_fact = FactSet(facts=facts, prior_p=prob, ground_true=2, len_list=len_list)
random_fact = FactSet(facts=facts, prior_p=prob, ground_true=2, len_list=len_list)
brute_fact = FactSet(facts=facts, prior_p=prob, ground_true=2, len_list=len_list)

max_cost:  6 note that budget of each round must > 18 and < 51:


## selector to select correspondence

### 初始化 selector

In [463]:
from query import QuerySelector, BaseQuerySelector, GreedyQuerySelector,RandomQuerySelector
 # 对应fact1, 3是0.8, 0.
query_selector = GreedyQuerySelector()
# selection_idxes, sub_facts, h = query_selector.select(ex_fact, 2, accuracy, cost_func=2)
random_selector = RandomQuerySelector()
base_selector = BaseQuerySelector()

## LLM答案dict
## 由于数量问题，模拟实验，从web chatgpt端，得到所有的correpspondence 验证答案

In [464]:
import json
with open(answers_path, "r") as f:
    ans_list = json.load(f)

### 参数设置

In [465]:
## params
# correspondence num

api_use = False 
c_len = ex_fact.num_fact()

#LLM 预测准确率
p_w = 0.9
# 计算p(A_T) 和 P(A_T|v) 需要的LLM array:acc 
acc = np.array([[p_w for i in range(c_len)]])

# 每个token 一个
budget = 40 # 单位为 price of one token  
turns = 20
##########################################

In [466]:
import numpy as np

approx_h_list=[ex_fact.compute_entropy()]

cost_sum = 0
while turns>0:
    selection_idxes, sub_facts, h = query_selector.select(ex_fact, budget, acc, cost_func=2)
    if api_use:
         ans = [1 if gpt_check(ix_r, c_set)=="yes" else 0 for ix_r in selection_idxes]
    else:
        ans = [1 if ans_list[ix_r]=="yes" else 0 for ix_r in selection_idxes]
    p_a,p_a_v = ex_fact.compute_ans_p(ans, selection_idxes, acc)
    sum_p = []
    for idx,i in enumerate(ex_fact.get_prior_p()):
        p_post = i*p_a_v[idx]/p_a
        sum_p.append(p_post)
    ex_fact.set_prior_p(np.array(sum_p))
    approx_h_list.append(ex_fact.compute_entropy())
    turns -=1

In [467]:
all_h_list = []
cost_list = []
p_prior = random_fact.get_prior_p()
for _ in range(100):
    cost_sum_r = 0
    random_fact.set_prior_p(p_prior)
    turns_r=20
    random_h_list=[random_fact.compute_entropy()]
    random_selection_idxes, _, _  =  random_selector.select(random_fact, budget, acc)
    while turns_r>0:
        # for ix_r in random_selection_idxes:
        #     cost_sum_r += ex_fact.len_list()[ix]
        #     ans_r = [1 if ans_list[ix_r]=="yes" else 0]
        #     p_a_r, p_a_v_r = random_fact.compute_ans_p(ans_r, [ix_r], acc)
        #     sum_p_r = []
        #     for idx,i in enumerate(random_fact.get_prior_p()):
        #         p_post_r = i*p_a_v_r[idx]/p_a_r
        #         sum_p_r.append(p_post_r)
        #     random_fact.set_prior_p(np.array(sum_p_r))
        if api_use:
            ans_r = [1 if gpt_check(ix_r, c_set)=="yes" else 0 for ix_r in random_selection_idxes]
        else:
            ans_r = [1 if ans_list[ix_r]=="yes" else 0 for ix_r in random_selection_idxes]
        p_a_r,p_a_v_r = random_fact.compute_ans_p(ans_r, random_selection_idxes, acc)
    
        sum_p_r = []
        for idx,i in enumerate(random_fact.get_prior_p()):
            p_post_r = i*p_a_v_r[idx]/p_a_r
            sum_p_r.append(p_post_r)
        random_fact.set_prior_p(np.array(sum_p_r))
        random_h_list.append(random_fact.compute_entropy())
        turns_r -=1
    all_h_list.append(random_h_list)
    cost_list.append(cost_sum_r)

In [468]:
approx_h_list

[2.0179569806623436,
 0.06647723876515912,
 0.00020312553145128786,
 3.4248007772491767e-06,
 5.362396693911432e-08,
 9.689744258836293e-11,
 1.64961660325674e-13,
 2.239756002390407e-15,
 3.090440380925279e-17,
 4.216975178650379e-19,
 5.701965262137485e-21,
 7.651590528901612e-23,
 1.0202120705202144e-24,
 1.3528189321027066e-26,
 1.7853293697396283e-28,
 2.3463109996578138e-30,
 3.072236625066345e-32,
 4.009620992150077e-34,
 5.9461253730027415e-37,
 7.707940298336882e-39,
 9.969117375861922e-41]

In [469]:
cost_sum

0

In [470]:
import numpy as np
n = np.array(all_h_list)
random_h_l = n.mean(axis=0, keepdims=True)
random_h_l

array([[2.01795698, 0.32667247, 0.18802263, 0.16491988, 0.16134928,
        0.16083871, 0.16076929, 0.16076016, 0.16075899, 0.16075885,
        0.16075883, 0.16075882, 0.16075882, 0.16075882, 0.16075882,
        0.16075882, 0.16075882, 0.16075882, 0.16075882, 0.16075882,
        0.16075882]])

In [471]:
import json
dataset = "output"
def save_h_file(approx_name_f,random_f, approx_ob, random_ob):
    with open(f"./{dataset}/"+approx_name_f, "w") as f:
        json.dump(approx_ob, f, indent=2, ensure_ascii=False)
    with open(f"./{dataset}/"+random_f, "w") as w:
        json.dump(random_ob, w, indent=2, ensure_ascii=False)

In [472]:
name = "turns_20_budget={}_{}"
save_h_file(name.format(budget,f"{dataset_name}_approx.json"), name.format(budget,f"{dataset_name}_random.json"),approx_h_list, random_h_l.tolist()[0])